In [1]:
import pandas as pd
import numpy as np
from tabulate import tabulate

import os
import psycopg2

from dotenv import load_dotenv
import logging
from tqdm import tqdm
import multiprocessing
import time

from pharmacy_common import PharmacyCommon
#Rdkit ultis
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem import MACCSkeys

import sys
sys.path.append("/home/mylab-pharma/Code/tuele/pan_HDAC/mylab_panHDAC-master/src/common")

#class to encode smiles
common = PharmacyCommon()

In [2]:
def cal_tc(array1, array2):
    if len(array1) != len(array2):
        raise ValueError("The arrays must have the same length.")
    
    # Calculate the Tanimoto coefficient
    intersection = sum(a and b for a, b in zip(array1, array2))
    union = sum(a or b for a, b in zip(array1, array2))
    
    if union == 0:  # Handle the case when both arrays are all zeros
        return 0.0
    else:
        tanimoto_coefficient = intersection / union
        return tanimoto_coefficient


In [3]:
def calculate_distances(identifier, screening_vectors, X_Train):
    distances = []
    for screening_vector in screening_vectors:
        tc_dist = np.sum(screening_vector & X_Train, axis=1) / np.sum(screening_vector | X_Train, axis=1)
        distances.append(tc_dist)
    return identifier, distances

def find_nearest_neighbors_distance(X_Screening, X_Train, n_neighbors):
    nearest_neighbors_distances = []
    nearest_neighbors_indices = []
    X_Screening = np.array(X_Screening)
    X_Train = np.array(X_Train)
    if(np.size(X_Screening) == 0):
        return nearest_neighbors_distances, nearest_neighbors_indices
     
    if X_Screening.shape[1] != X_Train.shape[1]:
        raise ValueError("X_Screening bit vectors must have the same size as X_Train bit vectors: " + str(X_Train.shape[1]))

    num_processes = 6
    
    if len(X_Screening) <= num_processes:
        screening_chunks = [(i, X_Screening[i:i + 1]) for i in range(len(X_Screening))]
    else:
        chunk_size = len(X_Screening) // num_processes
        screening_chunks = [(i, X_Screening[i:i + chunk_size]) for i in range(0, len(X_Screening), chunk_size)]

    pool = multiprocessing.Pool(processes=num_processes)
    results = pool.starmap(calculate_distances, [(i, chunk, X_Train) for i, chunk in screening_chunks])
    pool.close()
    pool.join()

    # Sort the results by identifier to ensure the correct order
    results.sort(key=lambda x: x[0])

    # Extract the distances and indices
    for _, distances in results:
        for distance in distances:
            # Get the indices of the first n_neighbors elements with the largest Tanimoto coefficients
            nearest_neighbor_indices = np.argsort(distance)[::-1][:n_neighbors]

            # Extract the distances to the nearest neighbors
            nearest_neighbors_distances.append([distance[i] for i in nearest_neighbor_indices])
            nearest_neighbors_indices.append(nearest_neighbor_indices)

    return nearest_neighbors_distances, nearest_neighbors_indices

In [4]:
def update_average_distance(screening_data, n_neighbors, X_train, bits):
    logging.info(f"[+] Update average distance for screening dataset")
    working_dataset = screening_data
    if(len(working_dataset)>0):
        X_Screening = common.gen_ecfp6_fpts(working_dataset["SMILES"], bits)
        logging.info(f"[-] Start finding nearest neighbor for: {len(X_Screening)}")
        #Find nearest neighbor
        dist_array, nn_idx = find_nearest_neighbors_distance(X_Screening=X_Screening, n_neighbors=n_neighbors, X_Train=X_train)
        result_df = pd.DataFrame({'SMILES': working_dataset['SMILES'], 
                                 'AVG_DISTANCE': np.average(dist_array, axis=1)})
        # for idx, row in working_dataset.iterrows():
        #     smiles = row['SMILES']
        #     nn_dist = dist_array[idx]
        #     #Calculate average distance
        #     avg_distance = np.average(nn_dist)
        #     # CHU Y O DAY
        #     result_df = pd.concat([result_df, pd.DataFrame({'SMILES': [smiles], 'AVG_DISTANCE': [avg_distance]})])
    else:
        logging.info("[-] Empty data, skip this batch!")
        print("[-] Empty data, skip this batch!")    
    return result_df

Split all screen_data with 25000 smiles

In [5]:
screening_data_path = "../../data/screening_data/all_screen_data.xlsx"
screening_data = pd.read_excel(screening_data_path, sheet_name='final_screen_data')

In [6]:
len(screening_data)

542869

In [7]:
# Tính tổng số hàng trong tập dữ liệu
total_rows = len(screening_data)

# Tính số lượng tập con cần chia
num_splits = (total_rows // 50000) + (1 if total_rows % 50000 != 0 else 0)

# Chia tập dữ liệu thành các tập con
data_splits = []
for i in range(num_splits):
    start = i * 50000
    end = min((i + 1) * 50000, total_rows)
    data_splits.append(screening_data.iloc[start:end])
# Bây giờ bạn có thể sử dụng chúng theo cách mình muốn
for i, split in enumerate(data_splits):
    print(f"Tập dữ liệu con {i+1} có {len(split)} hàng")

Tập dữ liệu con 1 có 50000 hàng
Tập dữ liệu con 2 có 50000 hàng
Tập dữ liệu con 3 có 50000 hàng
Tập dữ liệu con 4 có 50000 hàng
Tập dữ liệu con 5 có 50000 hàng
Tập dữ liệu con 6 có 50000 hàng
Tập dữ liệu con 7 có 50000 hàng
Tập dữ liệu con 8 có 50000 hàng
Tập dữ liệu con 9 có 50000 hàng
Tập dữ liệu con 10 có 50000 hàng
Tập dữ liệu con 11 có 42869 hàng


In [8]:
train_test_path = "../../data/train_test_data/NoCL/20240321_pan_HDAC_train_test_data.xlsx"
train_dataset = pd.read_excel(train_test_path, sheet_name='train_dataset')
#Fingerprint ECFP4, ECFP6
len(train_dataset)

1528

In [9]:
all_result = pd.DataFrame()

In [50]:
# tap thu i, co 19 tap : tu 0 -> 10
i = 10
screen_dataset = data_splits[i] 

In [51]:
fpt_bits = 2048
X_Train = common.gen_ecfp6_fpts(train_dataset['SMILES'], bits= fpt_bits)
# nn_nums: numbers of nearest-neighbors
## AD for ECFP4 2048bits, ECFP6 2048bits, ECFP4 1024bits
nn_nums = 5
result= update_average_distance(screening_data=screen_dataset, n_neighbors=nn_nums,X_train = X_Train, bits= fpt_bits)

Progress:   0%|          | 0/1528 [00:00<?, ?it/s]

[11:35:28] Conflicting single bond directions around double bond at index 7.
[11:35:28]   BondStereo set to STEREONONE and single bond directions set to NONE.
[11:35:28] Conflicting single bond directions around double bond at index 16.
[11:35:28]   BondStereo set to STEREONONE and single bond directions set to NONE.
[11:35:28] Conflicting single bond directions around double bond at index 33.
[11:35:28]   BondStereo set to STEREONONE and single bond directions set to NONE.
[11:35:28] Conflicting single bond directions around double bond at index 18.
[11:35:28]   BondStereo set to STEREONONE and single bond directions set to NONE.
[11:35:28] Conflicting single bond directions around double bond at index 27.
[11:35:28]   BondStereo set to STEREONONE and single bond directions set to NONE.
[11:35:28] Conflicting single bond directions around double bond at index 9.
[11:35:28]   BondStereo set to STEREONONE and single bond directions set to NONE.
[11:35:28] Conflicting single bond directi

In [52]:
result.head(10)

,SMILES,AVG_DISTANCE
500000,C1=CC=C(C(=C1)F)SCC(=O)NOCC(=O)N,0.187057
500001,C1=CC(=C(C(=C1[N+](=O)[O-])F)C(=O)NOCC(=O)N)F,0.186373
500002,C1=CC(=C(C=C1F)C(=O)NOCC(=O)N)[N+](=O)[O-],0.188059
500003,C1CC(CCC1C(=O)NOCC(=O)N)C(F)(F)F,0.137738
500004,C1=CC(=C(C=C1C(=O)NOCC(=O)N)[N+](=O)[O-])F,0.221519
500005,C1=CC(=CC(=C1)F)OCCC(=O)NOCC(=O)N,0.227585
500006,C1=CC=C(C(=C1)C(=O)NOCC(=O)N)SC(F)(F)F,0.204366
500007,C1=C(C(=CC(=C1[N+](=O)[O-])F)F)C(=O)NOCC(=O)N,0.172333
500008,C1=CC(=CC=C1OCCC(=O)NOCC(=O)N)F,0.239995
500009,C1=C(C(=CC(=C1F)F)[N+](=O)[O-])C(=O)NOCC(=O)N,0.173676


In [53]:
# all_result = result
all_result = pd.concat([all_result ,result],ignore_index=True, axis =0)
print(len(all_result))
all_result.head(10)

542869


,SMILES,AVG_DISTANCE
0,O=C1CCON1,0.105581
1,CCC(=O)NO,0.241693
2,NCC(=O)NO,0.216708
3,CON(C)C(C)=O,0.159957
4,ON1C(=O)CCC1=O,0.106835
5,CC1(C)CONC1=O,0.111817
6,CC(C)(C)C(=O)NO,0.189907
7,CCC(=O)N(C)OC,0.181227
8,CCC(N)C(=O)NO,0.223244
9,COCONC(C)=O,0.172408


In [55]:
screen_dataset_ecfp6_2048 = all_result[all_result["AVG_DISTANCE"]>0.5]
len(screen_dataset_ecfp6_2048)

297

In [55]:
# Lưu tất cả vào file excel 
all_result.to_excel(f'../../results/avg_distance/20240416_average_distance_ecfp6_{fpt_bits}.xlsx', index=False)
logging.info("[-] Finished finding, saved average distance to Excel!")
logging.info("\n")